# Monthly Ice Rates of Change

Compute the Change in ice concentration month over month.

Walt: "I would do it by taking extent on last day of month minus the last day of the previous month. Then divide by # of days in month to get daily rate.  I used the 5-day values."

In [ ]:
import calendar

import numpy as np
import pandas as pd
from pandas import ExcelWriter

import seaice_timeseries.daily as daily

In [ ]:
km2_2_mi2 = 0.386102

In [ ]:
month_dict = {}
for i in range(1,13):
    month_dict[i] = calendar.month_name[i]

In [ ]:
def rolling_mean(df, column, num_days=5, min_valid=2):
    return pd.rolling_mean(df[column], window=num_days, min_periods=min_valid)


In [ ]:
df_in = daily.daily_data('north')

In [ ]:
df = df_in.copy()
df['interpolated_extent'] = df['extent'].interpolate(limit=1)
df['5 Day'] = rolling_mean(df, 'interpolated_extent')
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

In [ ]:
a = df.groupby([df.index.year, df.index.month]).last()
a['ice change Mkm^2 per month'] = a['5 Day'].diff(periods=1)
a['ice change km^2 per day'] = (a['ice change Mkm^2 per month'] / a['day']) * 1000000
a['ice change mi^2 per month'] = a['ice change Mkm^2 per month'] * km2_2_mi2 * 1000000
a['ice change mi^2 per day'] = a['ice change km^2 per day'] * km2_2_mi2

In [ ]:
a.loc[2015, 6]

In [ ]:
a = a.unstack(1)

In [ ]:
a = a.rename(columns=month_dict)

In [ ]:
a.head()

In [ ]:
writer = ExcelWriter('../output/Sea_Ice_Rates_of_Change.xls', engine='xlsxwriter')

sheets = [u'ice change Mkm^2 per month', u'ice change km^2 per day', u'ice change mi^2 per month', u'ice change mi^2 per day']

for sheet in sheets:
    a[sheet].to_excel(writer, "{0}".format(sheet), float_format="%.3f", startrow=1, startcol=0)
    writer.sheets[sheet].write_string(0,0, "{} from 5 Day Averaged Daily Values".format( sheet ))
    

workbook = writer.book
# add colors blue with blue
format1 = workbook.add_format({'bg_color':   '#CEC7FF',
                               'font_color': '#06009C'})

# add colors red with red
format2 = workbook.add_format({'bg_color':   '#FFC7CE',
                               'font_color': '#9C0006'})



for sheet in sheets:
    worksheet = writer.sheets[sheet]
    worksheet.conditional_format('B3:ZZ369', {'type':     'cell',
                                              'criteria': '>',
                                              'value':    0,
                                              'format':   format1})

    worksheet.conditional_format('B3:ZZ369', {'type':     'cell',
                                              'criteria': '<',
                                              'value':    0,
                                              'format':   format2})



writer.save()